In [76]:
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt
from numba import jit
import ipywidgets as wid
from ipywidgets import interactive,fixed, interactive_output
import numpy.polynomial.polynomial as poly

RY=13.60566 #x*RY: Rydberg -> ev
ang=1.88972 #X/ang: raio de bohr -> angstroms
pot_exemplos=['(2e-4)*((x)**2)','0',
             '(1e-5)*(x**2)+(1e-7)*(x**4)',
             '(10)*( (49.2 / (x/ang+84) )**12 - (49.2 / (x/ang+84) )**6)']
def diagg(boo):
    global x,psi,prob,eig,V,L,N
    DIAG.description='Diagonalizando...'
    
    @jit
    def diagH(H,V):
        H=H+V
        eig, psi=np.linalg.eigh(H) #Resolve o sistema e atribui-se: eig=autovalores, psi=autovetores
        prob=np.abs(psi)**2 #Atribui a densidade de probabilidade |Psi|² a variável prob
        return H,V,psi,prob,eig
    
    N=x.size #Armazena o tamanho do vetor posição e o comprimento do espaço que ele representa
    a=- 1/(dx**2) #Define 'a' por conveniência
    H=sparse.diags([a,-2*a,a],[-1,0,1],shape=(N,N),dtype=np.complex128).toarray() #Cria a matriz Hamiltoniano
    H,V,psi,prob,eig=diagH(H,V)
    
    V=np.diag(V)
    PLOT.children[8].disabled=False
    PLOT.children[8].description=('Plotar')
    DIAG.disabled=True
    DIAG.description='Diagonalizado!'
    tab.selected_index=1
    
def syst(xx,N,potV):   
    global x,dx,V,Vlim
    def vetores(xx,N):
        xmin,xmax=xx
        x,dx=np.linspace(xmin*ang,xmax*ang, N, 
                         retstep=True,dtype=np.float64) #Define o vetor posição (raio de bohr)
        return x,dx,xmin,xmax
    
    def pot(N,pot):
        V=np.ones(shape=(N))
        V=V*eval(pot)/RY
        V=np.diag(V)
        return V
    
    x, dx, xmin, xmax=vetores(xx,N)
    V=pot(N,potV)
    
    xmin, xmax= x.min()/ang, x.max()/ang
    L=xmax-xmin
    Vlim=[V.min()*RY, V.max()*RY]
    info.children[1].value=f"{dx/ang: .2f}"
    info.children[3].value=f"{L: .2f}"
    info.children[5].value=f"[{Vlim[0]:.5f} , {Vlim[1]:.5f}]"
    info.children[7].value=SYST.children[2].value
    info.children[9].value=f"{SYST.children[1].value: 1d}"
    
    fig1, ax1= plt.subplots(figsize=(12,8))
    ax1.set_xlabel('$x (\AA)$')
    ax1.set_ylabel('Potencial (eV)')  
    
    plt.plot(x/ang,np.diag(V)*RY,'r-',label='$V(x)$')    
    plt.axhline(Vlim[0], xmin, xmax,color='r',ls='--',label='limites de $V(x)$')
    plt.axhline(Vlim[1], xmin, xmax,color='r',ls='--')
    plt.axvline(x=x.min()/ang,color='k',ls='--',label='limites de $x$')
    plt.axvline(x=x.max()/ang,color='k',ls='--',)
    plt.setp(ax1, xticks=[xmin, xmax], yticks=Vlim)
    leg=plt.legend(loc='upper right')
    
    DIAG.disabled=False
    DIAG.description='Diagonalizar'
    PLOT.children[8].disabled=True
    PLOT.children[8].description=('Diagonalize...')
    INFO.selected_index=0
    return V,x,dx,xmin,xmax
    
def plotall(NES,limx,wav,xs,ys,dpi,limy,autolim):
    fig, ax = plt.subplots(figsize=(xs,ys), dpi=dpi)
    nmin,nmax = NES
    En=eig*RY
    #elim=(En.min(), En.max())###########
    ax.set_xlim([x.min()/ang-limx, x.max()/ang +limx])
    ax.set_xlabel('$x (\AA)$')
    ax.set_ylabel('Auto energias (eV)')
    plt.title(f"$\Delta x$ = {dx/ang:.2f}, Ordem da matriz = {N} x {N}")
    probb=norm(prob,nmin,nmax,En,1)
    if wav==1:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x/ang,probb[:,i]+En[i], 
                         label=f'$E_{{{i}}}={En[i]:.5f}$ eV')
    elif wav==2:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x/ang,np.real(psi[:,i])+En[i], 
                         label=f'$E_{{{i}}}={En[i]:.5f}$ eV')
    elif wav==3:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x/ang,np.imag(psi[:,i])+En[i], 
                         label=f'$E_{{{i}}}={En[i]:.5f}$ eV')
    if autolim==False:
        ax.set_ylim(limy)
        
    fig=plt.axvline(x=x.min()/ang,color='k',ls='--')
    fig=plt.axvline(x=x.max()/ang,color='k',ls='--')
    fig=plt.plot(x/ang,V*RY,'k',label='V($x$)')
    leg=plt.legend(loc='upper right')
    
    plt.show()

@jit
def norm(prob,nmin,nmax,En,wav):
    mmax=np.zeros(nmax+2)
    if wav==1:
        for i in range(0,nmax+2):
            mmax[i]=(En[i+1]-En[i])*0.9
        for i in range(0,nmax+2):
            prob[:,i]=prob[:,i]*np.abs(mmax[i]/prob[:,i].max())
    else:
        for i in range(1,nmax+2):
            mmax[i]=(prob[:,i+1].min()-prob[:,i-1].max() \
                    /(prob[:,i].max()-prob[:,i].min()))
        for i in range(1,nmax+2):
            prob[:,i-1]=prob[:,i-1]*np.abs(mmax[i]*0.9)
    return prob

def potlist(bo):
    b=bo.description
    if b=='Oscilador Harmônico':
        SYST.children[2].value=pot_exemplos[0]
    elif b=='Poço Infinito':
        SYST.children[2].value=pot_exemplos[1]
    elif b=='Oscilador Duffing Holmes':
        SYST.children[2].value=pot_exemplos[2]
    elif b=='Lennard Jones':
        SYST.children[2].value=pot_exemplos[3]
    SYST.children[0].value=[-35,35]
    SYST.children[3].click()
    
################################STYLES/LAYOUT###############################
lslider=wid.Layout(width='90%', display='flex')
sty = {'description_width': 'initial'}


################################INTERACTIVES################################
SYST=interactive(syst, {'manual': True}, 
                    xx=wid.FloatRangeSlider(
                        value=[-35.0, 35.0],
                        min=-150.0,
                        max=150.0,
                        step=1,
                        continuous_update=False,
                        orientation='horizontal',
                        readout_format='1d',
                        layout=lslider,
                        description=('Domínio de $\Psi(x)$: '),
                        style=sty),
                    N=wid.IntSlider(value=550,
                        min=50,
                        max=1050,
                        step=10,
                        continuous_update=False, 
                        layout=lslider,
                        description=('Número de pontos: '),
                        style=sty),
                    potV=wid.Text(description='Expressão de $V(x)$: ',
                        value="0",
                        placeholder='Escreva a expressão aqui.',
                        style=sty,
                        layout=lslider))

PLOT=interactive(plotall, {'manual': True},
                    NES=wid.IntRangeSlider(
                        value=[0, 6],
                        min=0,
                        max=20,
                        step=1,
                        description='Plotar estados:',
                        continuous_update=False,
                        orientation='horizontal',
                        layout=lslider,
                        style=sty),
                    limx=wid.IntSlider(
                        value=10,
                        min=0,
                        max=100,
                        step=1,
                        description='Margens horizontais  (angstroms):   ',
                        continuous_update=False,
                        layout=lslider,
                        style=sty),
                    wav=wid.Dropdown(
                        options=[('densidades de probabilidades',1),
                                 ('funções de onda (parte real, +)',2),
                                 ('funções de onda (parte imaginária)',3)],
                        value=1,
                        description='Plotar as',
                        style=sty),
                    xs=wid.IntSlider(value=9,
                        min=0,
                        max=20,
                        step=1,
                        description='Largura da imagem (polegadas):',
                        style=sty,
                        layout=lslider),
                    ys=wid.IntSlider(value=9,
                        min=0,
                        max=20,
                        step=1,
                        description='Altura da imagem (polegadas): ',
                        style=sty,
                        layout=lslider),
                    dpi=wid.IntSlider(value=80,
                        min=10,
                        max=300,
                        step=1,
                        description='Pixels por polegada:',
                        style=sty,layout=lslider),
                    limy=wid.FloatRangeSlider(value=[0.,5.],
                        min=-25.,
                        max=25.,
                        step=0.05,
                        description='Limites do eixo y (angstroms):',
                        orientation='horizontal',
                        readout_format='.1f',
                        layout=lslider,
                        style=sty),
                    autolim=wid.Checkbox(
                        value=True,
                        description='Auto limitar eixo y',
                        style=sty))

################################GRIDS################################
items = [wid.Label('Passo $\Delta x $ (angstroms) ='), wid.Label('dx'), 
        wid.Label('Largura do domínio (angstroms) ='), wid.Label('L'),
        wid.Label('V(x)[min,max] (eV) ='), wid.Label('[V_min,V_max]'),
        wid.Label('Expressão V(x) ='), wid.Label(''),
        wid.Label('Número de pontos ='), wid.Label('')]
info=wid.GridBox(items, layout=wid.Layout(grid_template_columns="repeat(2, 250px)"))

################################BOTOES################################
DIAG=wid.Button(description='Diagonalizar',disabled=True)
OHS=wid.Button(description='Oscilador Harmônico')
POCO=wid.Button(description='Poço Infinito')
DUFHOLMES=wid.Button(description='Oscilador Duffing Holmes', style=sty)
LENJON=wid.Button(description='Lennard Jones', style=sty)

################################ON_EVENTS###############################
DIAG.on_click(diagg)
OHS.on_click(potlist)
POCO.on_click(potlist)
DUFHOLMES.on_click(potlist)
LENJON.on_click(potlist)

################################BOXES###############################
POTSBUT=wid.Box([POCO,OHS,DUFHOLMES,LENJON])
POTLIST=wid.Accordion([POTSBUT])
INFODIAG=wid.VBox([info,DIAG])
INFO=wid.Accordion([INFODIAG])
POTLIST.set_title(0,'Potenciais pré definidos')
INFO.set_title(0,'Diagonalizar/Informações')
SYSTBOX=wid.VBox([POTLIST,SYST,INFO])

################################UI################################
SYST.children[3].description=('Gerar')
PLOT.children[8].description=('Plotar')
PLOT.children[8].disabled=True
INFO.selected_index=None
################################TABS################################
tab=wid.Tab(children=[SYSTBOX,PLOT])
tab.set_title(0, 'Gerar o Sistema')
tab.set_title(1, 'Funções de Onda')
display(tab)